mutual information(상호의존정보)이 클수록 많은 정보를 알려줌

동시발생행렬의 문제점 : the, car, drive를 비교하면 the car보다 car drive가 관련성이 더 높지만, the car의 동시발생회수가 일반적으로 더 많기때문에 문제가 된다(the는 의미를 가지지않는 고빈도 단어라 문제가 됨)

pointwise mutual information(상호정보량) : 위의 문제를 해결하기 위해서 PMI를 사용
식(PMI) : log P(x,y)/P(x) * P(y)
the, car에 대한 pmi 식 = log P(the, car)/P(the) * P(car)
car, drive에 대한 pmi 식 = log P(car, drive)/ P(car) * P(drive)
원래는 the가 나올 확률이 압도적으로 높으나 the가 car와 같이 쓰이는 빈도보다 문장에 나오는 the의 확률이압도적으로 많기 때문에 the,car PMI=< car,drive PMI가 성립됨  => 동시발생행렬 문제점 해결

positive pointwise mutual information(=pointwise mutual information변형)
PPMI(x,y) = max(0,PMI(x,y))   #단 pmi가 음수일때는 0으로 취급한다  #코퍼스가 길어질경우 효과적

차원이 크다 = 계산비용이 크다
차원축소 = 차원은 줄이되 중요한 데이터의 감소는 최소화(정보의 손실은 어쩔 수 없음)

직교행렬(Orthogonal Matrix) = A * At=I(단위행렬)  and  At * A=I일 때 직교행렬이라고 한다.
단위행렬(I) = 주대각선의 값이 모두 1, 나머지는 모두 0의 값을 가지는 행렬
참고 : https://wikidocs.net/24949

singular value decomposition(특이값 분해) = SVD
m * n 행렬 X는 U S Vt로 분해된다
U = m * m 직교행렬// S = n * n 직교행렬 // Vt = m * n 직사각 대각행렬

principal component analysis(PCA)
X = U * S * Vt  ==> XV = US 가 성립됨(V가 직교행렬이라면 역행렬=전치행렬이 되므로 Vt를 V로 넘길 수 있다)
여기에 적당한 크기로 열을 잘라내면 중요한 정보를 가급적 유지한 채 차원이 줄어든다
데이터를 행렬로 묶은 것을 X로 둠. 각 열의 평균을 0이라 하고
V의 열벡터는 사영(=yx+b의 형태에 각 점(자료)을 식으로 옮기는 것)시켰을 때 분산이 가장 커지는 축을 나타낸다.
(=ax+b 형태에서 가장 많은 점이 찍히도록 하는것이 분산이 가장 커지게 하는 것임)

In [1]:
#ppmi 구현 코드
def ppmi(C, verbose=False, eps = 1e-8):
    '''
    C : 동시발생 행렬 = > P(x)를 C(x)/N 으로 변환할 수 있음
    verbose : 진행상황 출력여부
    '''
    M = np.zeros_like(C, dtype=np.float32) # 동시발생행렬하고 사이즈가 똑같은 제로행렬만들기
                                           # np.zeros_like는 np.zeros와 같이 모든 변수를 0으로 채움 - 모양맞출때 no.zeros_like를 사용한다고 함
    N = np.sum(C)                          # 동시발생행렬의 합 => 확률분포로 만들어주기위해서 만들기(예시의 경우 14)
    S = np.sum(C, axis=0)                  # axis=0은 행을 뜻하는데, 동시발생행렬의 행을 합치라는 소리
    total = C.shape[0] * C.shape[1]        #예제의 경우는 7*7이므로 총 49가 나올 것  #[0]은 행, [1]은 열을 말하는듯
    cnt = 0
    
    for i in range(C.shape[0]):            # i는 행렬의 행
        for j in range(C.shape[1]):        # j는 행렬의 열
            pmi = np.log2(C[i, j] * N / (S[j]*S[i]) + eps) #위에서 적어놓은 변환과정에서 N은 소거가 가능
                                                            #eps는 분모가 0되는것을 방지하기위함
            M[i, j] = max(0, pmi)                           #ppmi 구하는 식

            if verbose:
                cnt += 1
                if cnt % (total//100 + 1) == 0:
                    print('%.1f%% 완료' % (100*cnt/total))
    return M